# spaCy's RegEx advanced (multi-wrod tokens, RegEx's finditer, and spans)

Based on **Dr. William Mattingly** video: https://www.youtube.com/watch?v=dIUTsFT2MeQ&t

and his Jupyter Book: http://spacy.pythonhumanities.com/02_06_complex_regex.html